In [1]:
import os
import numpy as np
# import working directory to check functions
os.chdir('/Users/schiend/Desktop/DOHERTY/R-workspace/cecelia/inst')

# config
import py.config_utils as cfg

%load_ext autoreload
%autoreload 2

In [2]:
base_dir = '/Volumes/Analysis_SSD/Dominik/cecelia/projects/8BR53W/ANALYSIS/'
zero_dir = os.path.join(base_dir, '0/SSmEIR/')
#im_path = os.path.join(zero_dir, 'ccidImage.ome.zarr')
im_path = os.path.join(zero_dir, 'ccidCorrected.zarr')
version_num = 1
task_dir = os.path.join(base_dir, str(version_num), 'SSmEIR')

In [3]:
from py.napari_utils import NapariUtils

napari_utils = NapariUtils()
napari_utils.viewer = None
napari_utils.open_viewer()
napari_utils.task_dir = task_dir

Could not find Java JRE compatible with x86_64 architecture


In [4]:
channel_names = [
    'EB', 'T-A', 'T-B', 'AF', 'gen'
]

In [5]:
napari_utils.open_image(
    im_path,
    use_channel_axis = True, as_dask = True,
    channel_names = channel_names,
    downsample_z = False
)

In [45]:
napari_utils.show_labels_all(
    show_labels = True,
    show_points = False,
    show_tracks = True,
    value_names = ['T', 'LV']
)

{'base': '/Volumes/Analysis_SSD/Dominik/cecelia/projects/8BR53W/ANALYSIS/1/SSmEIR/labels/T.zarr'}
{'base': '/Volumes/Analysis_SSD/Dominik/cecelia/projects/8BR53W/ANALYSIS/1/SSmEIR/labels/LV.zarr'}


In [9]:
import trimesh
from collections import Counter
import re
import pandas as pd
import numpy as np
from tqdm import tqdm

from py.label_props_utils import LabelPropsUtils
from py.pop_utils import PopUtils

import py.morpho_utils as morpho_utils
import py.slice_utils as slice_utils

import py.script_utils as script_utils
import py.config_utils as cfg

In [10]:
pops_a = ['T/tracked']
pops_b = ['LV/tracked']

In [11]:
# init pop utils
pop_utils = PopUtils()

In [12]:
# prepare bbox and centroid axis
bbox_axis = ['x', 'y', 'z']
centroid_axis = bbox_axis

# prepare property columns to get
prop_cols = [f'bbox_{side}_{axis}' for side in ['min', 'max'] for axis in bbox_axis]
prop_cols.extend([f'centroid_{axis}' for axis in centroid_axis])

In [13]:
pop_df_a = pop_utils.pop_df(
    task_dir,
    LabelPropsUtils(task_dir),
    'live',
    cols = prop_cols,
    pops = pops_a
)

In [14]:
# get contact populations
pop_df_b = pop_utils.pop_df(
    task_dir,
    LabelPropsUtils(task_dir),
    'live',
    cols = prop_cols,
    pops = pops_b
)


In [17]:
im_res = napari_utils.dim_utils.im_scale(
    as_dict = True, upper = False,
    dims = ['z', 'y', 'x']
)
im_res

{'z': 2.099992309064516, 'y': 1.516634765625, 'x': 1.516634765625}

In [20]:
# load meshes
meshes_a = morpho_utils.df_to_meshes(
    task_dir,
    pop_df_a, 'T',
    'centroid_t',
    [38],
    im_res = im_res, is_3D = True,
    add_value_name_to_name = False)
meshes_b = morpho_utils.df_to_meshes(
    task_dir,
    pop_df_b, 'LV',
    'centroid_t',
    [38],
    im_res = im_res, is_3D = True,
    add_value_name_to_name = False)

>> get meshes for centroid_t 38
>> get meshes for centroid_t 38


In [29]:
# add pop B to collision manager
m = trimesh.collision.CollisionManager()

for i, x in meshes_b.items():
    m.add_object(i, x)

In [48]:
m.min_distance_single(meshes_a[7486], return_name = True)[0]

4.776129944915805

In [36]:
# is object within another object?
inside_object = False

# use convex hull to test location
meshes_b[709].convex_hull.contains(meshes_a[7486].vertices).all()

Mesh is non-watertight for contained point query!


True

In [46]:
meshes_b[709].convex_hull.contains(meshes_a[7403].vertices).all()

False

In [43]:
meshes_a

{7331: <trimesh.Trimesh(vertices.shape=(290, 3), faces.shape=(576, 3))>,
 7354: <trimesh.Trimesh(vertices.shape=(304, 3), faces.shape=(604, 3))>,
 7357: <trimesh.Trimesh(vertices.shape=(358, 3), faces.shape=(712, 3))>,
 7358: <trimesh.Trimesh(vertices.shape=(140, 3), faces.shape=(276, 3))>,
 7393: <trimesh.Trimesh(vertices.shape=(360, 3), faces.shape=(716, 3))>,
 7394: <trimesh.Trimesh(vertices.shape=(280, 3), faces.shape=(556, 3))>,
 7395: <trimesh.Trimesh(vertices.shape=(182, 3), faces.shape=(360, 3))>,
 7398: <trimesh.Trimesh(vertices.shape=(286, 3), faces.shape=(568, 3))>,
 7403: <trimesh.Trimesh(vertices.shape=(412, 3), faces.shape=(820, 3))>,
 7408: <trimesh.Trimesh(vertices.shape=(368, 3), faces.shape=(732, 3))>,
 7409: <trimesh.Trimesh(vertices.shape=(282, 3), faces.shape=(560, 3))>,
 7410: <trimesh.Trimesh(vertices.shape=(258, 3), faces.shape=(512, 3))>,
 7412: <trimesh.Trimesh(vertices.shape=(265, 3), faces.shape=(530, 3))>,
 7417: <trimesh.Trimesh(vertices.shape=(156, 3), fa